In [1]:
# import modules
import pybamm
import numpy as np;import matplotlib.pyplot as plt;
from pybamm import constants,exp,tanh,sqrt;

from pybamm import exp,sqrt
def electrolyte_diffusivity_Nyman2008Constant(c_e, c_EC,T):
    D_c_e = (
        (c_e <= 2000) * (
            8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10) 
        + (c_e > 2000) * (
        8.794e-11 * (2000 / 1000) ** 2 - 3.972e-10 * (2000 / 1000) + 4.862e-10)
    )
    return D_c_e

def electrolyte_conductivity_Nyman2008Constant(c_e,c_EC, T):
    sigma_e = (c_e <= 2000) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)
    ) + (c_e > 2000) *  (
        0.1297 * (2000 / 1000) ** 3 - 2.51 * (2000 / 1000) ** 1.5 + 3.329 * (2000 / 1000)
    )
    return sigma_e
def electrolyte_conductivity_Nyman2008Constant_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    coff = 1  
    ratio = ( coff/2 + coff/2 *  tanh((x-4.541*0.5)*1.5))
    sigma_e = ratio * ((c_e <= 2000) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)
    ) + (c_e > 2000) *  (
        0.1297 * (2000 / 1000) ** 3 - 2.51 * (2000 / 1000) ** 1.5 + 3.329 * (2000 / 1000)
    ))
    return sigma_e
print(electrolyte_diffusivity_Nyman2008Constant(10,4541,298.15))
print(electrolyte_conductivity_Nyman2008Constant(10,4541,298.15))


4.82236794e-10
0.0307801297


In [2]:

# set-up and run model
V_max = 4.2;        V_min = 2.5;
total_cycle = 1; save_at_cycles= 1
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V (1 minutes period)" , )] * total_cycle ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 20 minutes (10 seconds period)" , )] * total_cycle ) 

Sim = [];Sol=[]; 

ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);

Para_0['Negative electrode porosity'] =  0.1
var = pybamm.standard_spatial_vars  
mesh_thick = 30; mesh_par=30;
var_pts = {
    var.x_n: int(mesh_thick),  
    var.x_s: int(20),  
    var.x_p: int(mesh_thick),  
    var.r_n: int(mesh_par),  
    var.r_p: int(mesh_par),  }  

pybamm.settings.tolerances['D_e__c_e'] = 10


In [3]:

# Run the first model, with Nyman2008 polynimial form
model = pybamm.lithium_ion.DFN(
    options={
        "solvent diffusion": "EC wo refill"     } )
c_e = model.variables["Electrolyte concentration [mol.m-3]"]
c_EC = model.variables["EC concentration [mol.m-3]"]
T = model.variables["Cell temperature [K]"]
model.variables["Electrolyte conductivity [S.m-1]"] =(
    Para_0['Electrolyte conductivity [S.m-1]'](c_e, c_EC,T))
model.variables["Electrolyte diffusivity [m2.s-1]"] =(
    Para_0['Electrolyte diffusivity [m2.s-1]'](c_e, c_EC,T))

sim = pybamm.Simulation(
    model, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),
    var_pts=var_pts,)  
try:
    sol = sim.solve(save_at_cycles=save_at_cycles,);
    print(sol.cycles[-1].steps[-1]);  # a way to check whether the solution is finalized 
except:
    print('Fail for electrolyte: Poly')
else:
    Sol.append(sol)   
    print('Succeed for electrolyte: Poly')


using EC wo refill for Li+
Succeed for electrolyte: Poly


In [4]:
# Run the second model, with Nyman2008 constant form
model_2 = pybamm.lithium_ion.DFN(
    options={
        "solvent diffusion": "EC wo refill"} )
Para_0.update({
    'Electrolyte conductivity [S.m-1]': 
    electrolyte_conductivity_Nyman2008Constant})
Para_0.update({
    'Electrolyte diffusivity [m2.s-1]': 
    electrolyte_diffusivity_Nyman2008Constant})
c_e = model_2.variables["Electrolyte concentration [mol.m-3]"]
c_EC = model_2.variables["EC concentration [mol.m-3]"]
T = model_2.variables["Cell temperature [K]"]
model_2.variables["Electrolyte conductivity [S.m-1]"] =(
    Para_0['Electrolyte conductivity [S.m-1]'](c_e,c_EC, T))
model_2.variables["Electrolyte diffusivity [m2.s-1]"] =(
    Para_0['Electrolyte diffusivity [m2.s-1]'](c_e,c_EC, T))

sim_2 = pybamm.Simulation(
    model_2, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),
    var_pts=var_pts,)  
try:
    sol_2 = sim_2.solve(save_at_cycles=save_at_cycles,);
    print(sol_2.cycles[-1].steps[-1]);  # a way to check whether the solution is finalized 
except:
    print('Fail for electrolyte: Constant')
else:
    Sol.append(sol_2)   
    print('Succeed for electrolyte: Constant')


using EC wo refill for Li+
Succeed for electrolyte: Constant


In [5]:
# Run the _3 model, with Nyman2008 constant_wEC form
model_3 = pybamm.lithium_ion.DFN(
    options={
        "solvent diffusion": "EC wo refill"} )
Para_0.update({
    'Electrolyte conductivity [S.m-1]': 
    electrolyte_conductivity_Nyman2008Constant_wEC})
Para_0.update({
    'Electrolyte diffusivity [m2.s-1]': 
    electrolyte_diffusivity_Nyman2008Constant})
c_e = model_3.variables["Electrolyte concentration [mol.m-3]"]
c_EC = model_3.variables["EC concentration [mol.m-3]"]
T = model_3.variables["Cell temperature [K]"]
model_3.variables["Electrolyte conductivity [S.m-1]"] =(
    Para_0['Electrolyte conductivity [S.m-1]'](c_e,c_EC, T))
model_3.variables["Electrolyte diffusivity [m2.s-1]"] =(
    Para_0['Electrolyte diffusivity [m2.s-1]'](c_e,c_EC, T))

sim_3 = pybamm.Simulation(
    model_3, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),
    var_pts=var_pts,)  
try:
    sol_3 = sim_3.solve(save_at_cycles=save_at_cycles,);
    print(sol_3.cycles[-1].steps[-1]);  # a way to check whether the solution is finalized 
except:
    print('Fail for electrolyte: Constant')
else:
    Sol.append(sol_3)   
    print('Succeed for electrolyte: Constant')


using EC wo refill for Li+
Succeed for electrolyte: Constant


In [6]:
print(Sol[0]["Electrolyte conductivity [S.m-1]"].entries[:,-1])
print(Sol[0]["Electrolyte concentration [mol.m-3]"].entries[:,-1])

[2.69161751 2.68217028 2.66327942 2.63495231 2.59720032 2.55003922
 2.49348972 2.42757827 2.35233817 2.26781119 2.17404988 2.07112088
 1.95910992 1.8381294  1.70833041 1.56992231 1.42320643 1.26863707
 1.10694086 0.9393737  0.7683563  0.599423   0.45017736 0.44868112
 0.7314624  0.89322823 0.94505846 0.94375355 0.90638659 0.84040497
 0.79529097 0.79339129 0.79148241 0.78956435 0.78763714 0.78570079
 0.78375535 0.78180082 0.77983724 0.77786462 0.77588299 0.77389237
 0.77189279 0.76988426 0.76786681 0.76584045 0.76380521 0.76176111
 0.75970816 0.7576464  0.74936156 0.73497477 0.72069946 0.70657964
 0.69265674 0.67896981 0.66555568 0.65244917 0.63968318 0.62728885
 0.61529564 0.60373142 0.59262252 0.58199381 0.57186871 0.56226928
 0.55321617 0.54472869 0.5368248  0.52952112 0.5228329  0.51677405
 0.51135709 0.50659318 0.50249206 0.49906208 0.49631013 0.49424166
 0.49286067 0.49216966]
[4453.34079024 4450.28989652 4444.16595578 4434.92398699 4422.49474181
 4406.78241596 4387.66133725 4364.

In [7]:
print(Sol[-1]["Electrolyte conductivity [S.m-1]"].entries[:,-1])
print(Sol[-1]["Electrolyte concentration [mol.m-3]"].entries[:,-1])

[0.00532659 0.00531895 0.00530372 0.00528098 0.00525087 0.00521356
 0.00516929 0.00511832 0.00506099 0.00499769 0.00492892 0.00485532
 0.00477774 0.00469746 0.00461651 0.00453845 0.00447    0.0044255
 0.00444221 0.00466771 0.00615849 0.01063063 0.02596515 0.1135425
 0.58037064 0.87995627 0.94549341 0.93946501 0.90038894 0.83586647
 0.79242493 0.79059911 0.78876397 0.78691953 0.7850658  0.78320278
 0.78133048 0.77944892 0.7775581  0.77565804 0.77374874 0.77183021
 0.76990246 0.76796551 0.76601935 0.764064   0.76209946 0.76012574
 0.75814286 0.75615081 0.74814449 0.73427779 0.72055414 0.70701329
 0.69369247 0.68062662 0.6678486  0.65538928 0.64327774 0.63154132
 0.62020578 0.60929532 0.59883266 0.5888391  0.57933459 0.57033771
 0.56186575 0.55393472 0.54655935 0.53975315 0.53352837 0.52789607
 0.52286607 0.51844701 0.51464633 0.51147027 0.50892392 0.50701118
 0.5057348  0.50509638]
[4087.93107763 4090.01637498 4094.19036554 4100.45943719 4108.83272716
 4119.32162891 4131.93889787 4146.69

In [11]:
fs=17; 
import matplotlib as mpl; 
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)
label = ['Nyman - Poly', 'Nyman - Constant', 'Nyman - Constant w EC', ]
output_variables3 = [
    "Terminal voltage [V]",   
    "Electrolyte potential [V]",
    "Electrolyte concentration [mol.m-3]",
    "Li+ flux [mol.m-2.s-1]",
    "Electrolyte diffusivity [m2.s-1]",
    "Electrolyte conductivity [S.m-1]"
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol], 
    output_variables3,label,
    variable_limits='tight',time_unit='minutes',n_rows=2,
    figsize = (12,8)) #     spatial_unit='mm',
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=58.87749847444097, step=0.5887749847444097),…

In [12]:

label = ['Nyman - Poly', 'Nyman - Constant', 'Nyman - Constant w EC', ]
output_variables3 = [
    "Terminal voltage [V]",
    "Positive electrode potential [V]",
    "Positive electrode current density [A.m-2]",
    "Electrolyte potential [V]",
    "Electrolyte current density [A.m-2]",
    "Sum of positive electrode interfacial current densities",
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol], 
    output_variables3,label,
    variable_limits='tight',time_unit='minutes',n_rows=2,
    figsize = (12,8)) #     spatial_unit='mm',
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=58.87749847444097, step=0.5887749847444097),…